In [1]:
# you get all pretrained model name here
# https://huggingface.co/transformers/pretrained_models.html
import pandas as pd
import os
import torch
import transformer
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
# from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, EarlyStoppingCallback
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [2]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
# model_name = "bert-base-uncased"
model_name = "distilbert-base-uncased"
model_dir = 'model_esxdeploy_m1_82'
# max sequence length for each document/sentence sample
max_length = 512

In [3]:
# tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [4]:
def read_passages(path_data, test_split=0.8):
    df_train = pd.read_csv(path_data)
    anchors_all = df_train['anchor'].to_list()
    documents_all = df_train['log'].to_list()
    labels_str_all = df_train['label'].to_list()
    
    samples_all = [(anchor, document) for anchor, document in zip(anchors_all,documents_all)]
    
    labels_index = sorted(list(set(labels_str_all)))
    index_labels = {l:idx for idx, l in enumerate(labels_index)}
    labels_all = [ index_labels[label_str] for label_str in labels_str_all]
    
    ids_sample = range(len(anchors_all))
    ids_train, ids_others, labels_train, labels_others = train_test_split(ids_sample, labels_all, stratify=labels_all, test_size=test_split)
    ids_eval, ids_test, labels_eval, labels_test = train_test_split(ids_others, labels_others, stratify=labels_others, test_size=0.5)
    
    samples_train = [samples_all[id_train] for id_train in ids_train]
    samples_eval = [samples_all[id_eval] for id_eval in ids_eval]
    samples_test = [samples_all[id_test] for id_test in ids_test]

    return samples_train, samples_eval, samples_test, labels_train, labels_eval, labels_test, labels_index
# end


In [5]:
# def read_passages(path_data_train, path_data_eval):
#     df_train = pd.read_csv(path_data_train)
#     anchers_train = df_train['ancher'].to_list()
#     documents_train = df_train['log'].to_list()
#     labels_str_train = df_train['label'].to_list()
    
#     df_eval = pd.read_csv(path_data_eval)
#     anchers_eval = df_eval['ancher'].to_list()
#     documents_eval = df_eval['log'].to_list()
#     labels_str_eval = df_eval['label'].to_list()
    

#     samples_train = [(ancher, document) for ancher, document in zip(anchers_train,documents_train)]
#     samples_eval = [(ancher, document) for ancher, document in zip(anchers_eval,documents_eval)]
    
#     labels_index = sorted(list(set(labels_str_train + labels_str_eval)))
#     labels_all = {l:idx for idx, l in enumerate(labels_index)}
    
#     labels_train = [labels_all[label_str] for label_str in labels_str_train]
#     labels_eval = [labels_all[label_str] for label_str in labels_str_eval]

#     len_eval_start = int(len(samples_eval) / 8)
#     len_eval_end = int(len(samples_eval) / 4)
#     print(f'{len_eval_start}:{len_eval_end}')
    
#     for id_item, item in enumerate(samples_eval):
#         if type(item[0]) != str or type(item[1]) != str:
#             print(id_item)
#         # end
#     # end

#     samples_eval = samples_eval[len_eval_start:len_eval_end]
#     labels_eval = labels_eval[len_eval_start:len_eval_end]
#     return samples_train, samples_eval, labels_train, labels_eval, labels_index
# # end

In [6]:
# call the function
dir_data = 'datasource'
filename = 'nimbus_trainingdataset_pair_20221207.csv'

path_file = os.path.join(dir_data, filename)

# samples_train, samples_eval, samples_test, labels_train, labels_eval, labels_test, labels_index
train_samples, valid_samples, test_samples, train_labels, valid_labels, test_labels, target_names = read_passages(path_file)

In [7]:
train_encodings = tokenizer.batch_encode_plus(train_samples, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
valid_encodings = tokenizer.batch_encode_plus(valid_samples, truncation=True, padding=True, max_length=max_length, return_tensors='pt')

In [8]:
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [9]:
train_dataset = SimpleDataset(train_encodings, train_labels)
valid_dataset = SimpleDataset(valid_encodings, valid_labels)


In [10]:
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))
if os.path.exists(model_dir) and len(os.listdir(model_dir)) > 0:
    print('load model from local')
    model_info = model_dir
else:
    print('load model from official')
    model_info = model_name
    
model = DistilBertForSequenceClassification.from_pretrained(model_info, num_labels=len(target_names))

load model from official


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids.reshape(-1)
    # pred = np.argmax(pred, axis=1)
    preds = pred.predictions.argmax(-1).reshape(-1)

    # print('labels: {}'.format(labels))
    # print('pred: {}'.format(preds))
    accuracy = accuracy_score(y_true=labels, y_pred=preds)
    precision = precision_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    recall = recall_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    f1 = f1_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [12]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=20,              # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=2,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=1,               # log & save weights each logging_steps
    evaluation_strategy="epoch",     # evaluate each `logging_steps`
    learning_rate=2e-5,
    save_strategy='epoch',
    save_total_limit=10,
    metric_for_best_model='f1'
)

In [13]:
# trainer = Trainer(
#     model=model,                         # the instantiated Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     train_dataset=train_dataset,         # training dataset
#     compute_metrics=compute_metrics,     # the callback that computes metrics of interest
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
# )

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

In [14]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1699
  Num Epochs = 20
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 17000
  Number of trainable parameters = 66955010
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.001600,0.276991,0.941453,0.863593,0.861314,0.862448
2,0.044600,0.356951,0.939100,0.906676,0.787993,0.833696
3,0.000500,0.211090,0.956164,0.907115,0.882205,0.894110
4,0.000100,0.291886,0.959106,0.920114,0.881793,0.899684
5,0.000100,0.275728,0.961753,0.927378,0.887473,0.906075
6,0.000100,0.238033,0.959106,0.923586,0.877620,0.898783
7,0.000100,0.294917,0.957929,0.905064,0.895728,0.900319
8,0.001400,0.314159,0.961459,0.915709,0.900867,0.908099
9,0.000000,0.302724,0.963224,0.929838,0.892483,0.909988
10,0.000000,0.331868,0.962930,0.928645,0.892315,0.909368


***** Running Evaluation *****
  Num examples = 3399
  Batch size = 2
Saving model checkpoint to ./results/checkpoint-850
Configuration saved in ./results/checkpoint-850/config.json
Model weights saved in ./results/checkpoint-850/pytorch_model.bin
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 3399
  Batch size = 2
Saving model checkpoint to ./results/checkpoint-1700
Configuration saved in ./results/checkpoint-1700/config.json
Model weights saved in ./results/checkpoint-1700/pytorch_model.bin
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than

TrainOutput(global_step=17000, training_loss=0.06323620198209504, metrics={'train_runtime': 1459.5062, 'train_samples_per_second': 23.282, 'train_steps_per_second': 11.648, 'total_flos': 4501242206330880.0, 'train_loss': 0.06323620198209504, 'epoch': 20.0})

In [15]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 3399
  Batch size = 2
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


{'eval_loss': 0.40676063299179077,
 'eval_accuracy': 0.9638128861429832,
 'eval_precision': 0.9253777671540767,
 'eval_recall': 0.9011634601505978,
 'eval_f1': 0.9127770773643751,
 'eval_runtime': 22.8334,
 'eval_samples_per_second': 148.861,
 'eval_steps_per_second': 74.452,
 'epoch': 20.0}

In [16]:
model.save_pretrained(model_dir)

Configuration saved in model_esxdeploy_m1_82/config.json
Model weights saved in model_esxdeploy_m1_82/pytorch_model.bin


In [18]:
import json
with open(f'{model_dir}/labels.json', 'w+') as file:
    file.write(json.dumps(target_names))

In [21]:
test_encodings = tokenizer.batch_encode_plus(test_samples, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
test_dataset = SimpleDataset(test_encodings, test_labels)

trainer2 = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)
trainer2.evaluate()

***** Running Evaluation *****
  Num examples = 3400
  Batch size = 2
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


{'eval_loss': 0.44405704736709595,
 'eval_accuracy': 0.9579411764705882,
 'eval_precision': 0.9122880517939483,
 'eval_recall': 0.8852991696868837,
 'eval_f1': 0.8981561487020775,
 'eval_runtime': 22.8147,
 'eval_samples_per_second': 149.027,
 'eval_steps_per_second': 74.513}